In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
h2o.init() 

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_144"; Java(TM) SE Runtime Environment (build 1.8.0_144-b01); Java HotSpot(TM) 64-Bit Server VM (build 25.144-b01, mixed mode)
  Starting server from /opt/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/tb/57r5vg6n2rjdg49kfqd_453m0000gp/T/tmp_nkrx9em
  JVM stdout: /var/folders/tb/57r5vg6n2rjdg49kfqd_453m0000gp/T/tmp_nkrx9em/h2o_yaseenh_started_from_python.out
  JVM stderr: /var/folders/tb/57r5vg6n2rjdg49kfqd_453m0000gp/T/tmp_nkrx9em/h2o_yaseenh_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Dubai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,2 months and 6 days
H2O_cluster_name:,H2O_from_python_yaseenh_5mdo6x
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.556 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


#Load the dataset  tobe trained

In [2]:
data=pd.read_csv("data_hashed_to_train_model.csv")
data.head()


,like,dtype,dconn,pos,A,B,C,D,E,F,...,sid_int,sdomain_int,scat_int,aid_int,adomain_int,acat_int,did_int,dip_int,dmodel_int,week_day_int
0,0,1,0,1,1005,20251,320,50,2323,0,...,7100358526439870984,-5310258484222950493,5972868601436526999,7805814409961345999,-776665081776717179,-957940335628360031,3470267876062906150,-424805730869772094,7870230495624822831,-9031220084145463596
1,0,0,0,0,1002,17894,320,50,2039,2,...,-7887201529777453713,-5613206317714066368,-1211157273377936692,7805814409961345999,-776665081776717179,-957940335628360031,2020784215500099333,6889482189471946590,8268809522513963383,6686300557777964987
2,1,1,2,0,1005,20345,300,250,2331,2,...,776321843154138061,3785538029929655034,-1211157273377936692,3488509226292039841,2910822173347745509,-2244377406479595259,3470267876062906150,-3792304322008459525,5191803448000548666,4935879207058060046
3,0,1,2,0,1005,22598,320,50,2608,3,...,776321843154138061,3785538029929655034,-1211157273377936692,6960301978751252913,2910822173347745509,-2244377406479595259,-868105548171413683,4108802060273207135,-7190707338329353789,5987037459740618995
4,0,1,0,0,1005,21611,320,50,2480,3,...,776321843154138061,3785538029929655034,-1211157273377936692,3283611302887031448,1199078864727329065,-2244377406479595259,4164311493201216310,-700808832967241347,-8260511977533243161,5987037459740618995


In [3]:
#splitting train and test data 75-25
from sklearn.model_selection import train_test_split
train,test = train_test_split(data,test_size = .25 ,random_state = 111)


In [4]:
train=h2o.H2OFrame(train)
test=h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
# Identify predictors and response
x = train.columns
y = "like"

x.remove(y)

In [6]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [7]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
%time
aml = H2OAutoML(max_models=20, seed=1,verbosity="info")
aml.train(x=x, y=y, training_frame=train,leaderboard_frame = test) #validation on test dataframe

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 12.9 µs
AutoML progress: |
11:12:16.380: Project: AutoML_20201017_111216377
11:12:16.384: Setting stopping tolerance adaptively based on the training frame: 0.0011547005383792516
11:12:16.385: Build control seed: 1
11:12:16.386: training frame: Frame key: automl_training_py_1_sid_bca8    cols: 25    rows: 750000  chunks: 46    size: 76691165  checksum: -1923162350720780720
11:12:16.386: validation frame: NULL
11:12:16.387: leaderboard frame: Frame key: py_2_sid_bca8    cols: 25    rows: 250000  chunks: 16    size: 25565234  checksum: -4854665538699318320
11:12:16.387: blending frame: NULL
11:12:16.387: response column: like
11:12:16.387: fold column: null
11:12:16.387: weights column: null
11:12:16.419: Loading execution steps: [{XGBoost : defaults}, {GLM : defaults}, {DRF : [def_1]}, {GBM : defaults}, {DeepLearning : defaults}, {DRF : [XRT]}, {XGBoost : grids}, {GBM : grids}, {DeepLearning : grids}, {GBM : [lr_annealing]}, {X

In [8]:
# View the AutoML Leaderboard performance based on testdataset
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20201017_111216,0.746608,0.402144,0.375982,0.323274,0.3539,0.125245
StackedEnsemble_BestOfFamily_AutoML_20201017_111216,0.745076,0.402852,0.374329,0.32307,0.354124,0.125404
GBM_5_AutoML_20201017_111216,0.74228,0.399712,0.369292,0.327101,0.353439,0.124919
GBM_grid__1_AutoML_20201017_111216_model_2,0.742255,0.399591,0.369064,0.327678,0.353518,0.124975
GBM_4_AutoML_20201017_111216,0.741407,0.400491,0.370173,0.325121,0.35357,0.125012
XGBoost_1_AutoML_20201017_111216,0.739707,0.400598,0.368146,0.325767,0.35378,0.125161
XGBoost_grid__1_AutoML_20201017_111216_model_4,0.739308,0.400981,0.366835,0.330667,0.353965,0.125291
XGBoost_3_AutoML_20201017_111216,0.738369,0.401481,0.364867,0.326745,0.354186,0.125448
XRT_1_AutoML_20201017_111216,0.737794,0.401599,0.36438,0.327218,0.354256,0.125498
XGBoost_grid__1_AutoML_20201017_111216_model_3,0.737333,0.401945,0.36441,0.328743,0.354293,0.125523


In [17]:
lb[:7]

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20201017_111216,0.746608,0.402144,0.375982,0.323274,0.3539,0.125245
StackedEnsemble_BestOfFamily_AutoML_20201017_111216,0.745076,0.402852,0.374329,0.32307,0.354124,0.125404
GBM_5_AutoML_20201017_111216,0.74228,0.399712,0.369292,0.327101,0.353439,0.124919
GBM_grid__1_AutoML_20201017_111216_model_2,0.742255,0.399591,0.369064,0.327678,0.353518,0.124975
GBM_4_AutoML_20201017_111216,0.741407,0.400491,0.370173,0.325121,0.35357,0.125012
XGBoost_1_AutoML_20201017_111216,0.739707,0.400598,0.368146,0.325767,0.35378,0.125161
XGBoost_grid__1_AutoML_20201017_111216_model_4,0.739308,0.400981,0.366835,0.330667,0.353965,0.125291
XGBoost_3_AutoML_20201017_111216,0.738369,0.401481,0.364867,0.326745,0.354186,0.125448
XRT_1_AutoML_20201017_111216,0.737794,0.401599,0.36438,0.327218,0.354256,0.125498
XGBoost_grid__1_AutoML_20201017_111216_model_3,0.737333,0.401945,0.36441,0.328743,0.354293,0.125523


In [19]:
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
model_ids

['StackedEnsemble_AllModels_AutoML_20201017_111216',
 'StackedEnsemble_BestOfFamily_AutoML_20201017_111216',
 'GBM_5_AutoML_20201017_111216',
 'GBM_grid__1_AutoML_20201017_111216_model_2',
 'GBM_4_AutoML_20201017_111216',
 'XGBoost_1_AutoML_20201017_111216',
 'XGBoost_grid__1_AutoML_20201017_111216_model_4',
 'XGBoost_3_AutoML_20201017_111216',
 'XRT_1_AutoML_20201017_111216',
 'XGBoost_grid__1_AutoML_20201017_111216_model_3',
 'GBM_3_AutoML_20201017_111216',
 'DRF_1_AutoML_20201017_111216',
 'XGBoost_grid__1_AutoML_20201017_111216_model_1',
 'GBM_2_AutoML_20201017_111216',
 'GBM_1_AutoML_20201017_111216',
 'GBM_grid__1_AutoML_20201017_111216_model_1',
 'XGBoost_2_AutoML_20201017_111216',
 'XGBoost_grid__1_AutoML_20201017_111216_model_2',
 'DeepLearning_1_AutoML_20201017_111216',
 'DeepLearning_grid__1_AutoML_20201017_111216_model_1',
 'DeepLearning_grid__2_AutoML_20201017_111216_model_1',
 'GLM_1_AutoML_20201017_111216']

In [27]:
# Get the Best performing model in terms of log_loss being lowest
se = h2o.get_model([mid for mid in model_ids if "GBM_5_AutoML_20201017_111216" in mid][0])


In [24]:
se.actual_params

{'model_id': 'GBM_5_AutoML_20201017_111216',
 'training_frame': 'automl_training_py_1_sid_bca8',
 'validation_frame': None,
 'nfolds': 5,
 'keep_cross_validation_models': False,
 'keep_cross_validation_predictions': True,
 'keep_cross_validation_fold_assignment': False,
 'score_each_iteration': False,
 'score_tree_interval': 5,
 'fold_assignment': 'Modulo',
 'fold_column': None,
 'response_column': 'like',
 'ignored_columns': [],
 'ignore_const_cols': True,
 'offset_column': None,
 'weights_column': None,
 'balance_classes': False,
 'class_sampling_factors': None,
 'max_after_balance_size': 5.0,
 'max_confusion_matrix_size': 20,
 'max_hit_ratio_k': 0,
 'ntrees': 55,
 'max_depth': 15,
 'min_rows': 100.0,
 'nbins': 20,
 'nbins_top_level': 1024,
 'nbins_cats': 1024,
 'r2_stopping': 1.7976931348623157e+308,
 'stopping_rounds': 0,
 'stopping_metric': 'logloss',
 'stopping_tolerance': 0.0011547005383792516,
 'max_runtime_secs': 0.0,
 'seed': 10,
 'build_tree_one_node': False,
 'learn_rate': 

##### The overall performance of GBM AutomL model on Train and Test dataset can be seen below

In [26]:
se.model_performance()


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.11917210410201887
RMSE: 0.3452131285192075
LogLoss: 0.38101913832078194
Mean Per-Class Error: 0.29047036737420906
AUC: 0.7856968755765652
AUCPR: 0.43708350933031453
Gini: 0.5713937511531304

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.23501484482047338: 


,,0,1,Error,Rate
0,0,494955.0,127476.0,0.2048,(127476.0/622431.0)
1,1,51245.0,76324.0,0.4017,(51245.0/127569.0)
2,Total,546200.0,203800.0,0.2383,(178721.0/750000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.235015,0.460659,209.0
1,max f2,0.134553,0.605372,289.0
2,max f0point5,0.305382,0.444094,156.0
3,max accuracy,0.425282,0.838505,94.0
4,max precision,0.873079,1.000000,0.0
5,max recall,0.016167,1.000000,398.0
6,max specificity,0.873079,1.000000,0.0
7,max absolute_mcc,0.235015,0.332335,209.0
8,max min_per_class_accuracy,0.199661,0.706865,237.0
9,max mean_per_class_accuracy,0.176332,0.709530,255.0



Gains/Lift Table: Avg response rate: 17.01 %, avg score: 17.03 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010000,0.580798,4.434463,4.434463,0.754267,0.655311,0.754267,0.655311,0.044345,0.044345,343.446292,343.446292,0.041384
1,2,0.020000,0.507792,3.612947,4.023705,0.614533,0.541843,0.684400,0.598577,0.036129,0.080474,261.294672,302.370482,0.072868
2,3,0.030000,0.455613,3.223354,3.756921,0.548267,0.479968,0.639022,0.559041,0.032234,0.112708,222.335364,275.692109,0.099659
3,4,0.040000,0.417356,3.003081,3.568461,0.510800,0.435395,0.606967,0.528129,0.030031,0.142738,200.308069,256.846099,0.123795
4,5,0.050000,0.389722,2.815731,3.417915,0.478933,0.402684,0.581360,0.503040,0.028157,0.170896,181.573109,241.791501,0.145674
5,6,0.100000,0.323568,2.510014,2.963965,0.426933,0.351265,0.504147,0.427153,0.125501,0.296396,151.001419,196.396460,0.236648
6,7,0.150000,0.289739,2.064922,2.664284,0.351227,0.305524,0.453173,0.386610,0.103246,0.399643,106.492173,166.428364,0.300807
7,8,0.200001,0.264109,1.782517,2.443838,0.303192,0.276564,0.415677,0.359098,0.089128,0.488771,78.251720,144.383762,0.347954
8,9,0.300000,0.223613,1.463463,2.117050,0.248923,0.243131,0.360093,0.320443,0.146344,0.635115,46.346283,111.705038,0.403798
9,10,0.400000,0.182291,1.120178,1.867832,0.190533,0.203532,0.317703,0.291215,0.112018,0.747133,12.017810,86.783231,0.418279


In [28]:
se.summary()


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,55.0,55.0,733035.0,0.0,15.0,10.909091,1.0,2030.0,1057.1273
